# 树

## 定义问题

当我们说“树形结构”的时候，那是从自然界的大树上得到的启发，从根开始不断展开的枝丫，代表了从单一到复杂的一层层展开，在现实世界充满了这样的事物：国家的行政区划（以及任何组织的架构）、电商网站的商品分类、论坛里的板块等等。

<img src="assets/tree-graph.png" width="700">

> 虽然自然界里树是根在下面而向上分支，但是我们在处理树形结构时，如图这样的从上向下分支更容易画也更容易看。

解决问题的第一步是先要清晰明确地定义问题，我们来尝试把我们对树的直觉印象书写成尽可能清晰明确的定义和表述。在一个树形结构里：
* 树（*tree*）由节点（*node*）和连接节点的边（*edge*）组成；
* 总有一个节点是分支的起点，它分出了所有其他的节点，这个节点叫根节点（*root node*）；
* 一个节点分支出来的节点叫它的“子节点（*child nodes*）”，它是其子节点的“父节点（*parent node*）”；上图中节点 A 的父节点是根节点，子节点是 B 和 C，而节点 C 的父节点是 A，子节点是 D 和 E；节点 D 的父节点是 C，而它没有子节点；
* 拥有共同父节点的两个节点互为“兄弟姐妹（*sibling nodes*）”；比如上图中 B 和 C，D 和 E；
* 没有子节点的节点也叫“叶子节点（*leaf node*）”；比如上图中的 D 和 E；
* 一个节点的父节点，以及父节点的父节点…直至根节点，都是这个节点的“祖先节点（*ancestor nodes*）”；比如上图中 E 的祖先节点包括 C、A 和 根节点；
* 一个节点的所有子节点，以及所有子节点的子节点…直至叶子节点，都是这个节点的“后代节点（*descendant nodes*）”；
* 一个节点 X 和它所有后代节点、以及这些节点之间的边，也组成一棵树，叫做原数的一棵（以 X 为根的）“子树（*sub-tree*）”；
* 根节点没有父节点也没有祖先节点；
* 除了根节点以外的任何节点，有且只有 1 个父节点，有至少 1 个祖先节点；
* 任何节点都可以有 0、1 或者多个子节点，可以有  0、1 或者多个后代节点；
* 一个节点和它的任一祖先或者后代节点之间，一定存在一条由边收尾连接组成的路径（*path*），比如上图中根节点是 E 的祖先，它们之间的路径是 `root -> A -> C -> E`；这个路径就像是两个节点之间的亲属关系链；
* 两个节点之间 *path* 的长度，就是它由几条边组成，决定了这两个节点之间隔了多少“代”，也叫节点之间的“距离（*distance*）”；
* 一个节点和根节点之间的 *distance* 经常有特别的含义，相当于该节点在树的“第几层级（*tier*）”；比如在行政区划里哪些是一级节点（省、直辖市、自治区）哪些是二级节点等；
* 有些树里节点的子节点是有顺序的，叫做“有序树（*ordered tree*）”；如果我们不特别指出，那就是不考虑这种顺序概念。

上面这种“把直觉规则化”的过程，对后面设计解决方案是至关重要的，大家可以自己尝试，多多体会。

> 事实上在数学图论里有对树的[更严谨更数学化的定义](https://en.wikipedia.org/wiki/Tree_(graph_theory))，不过上面的表述对目前的我们来说基本够用了。

## 分析操作场景

可以看到树和我们前面讲的所有数据结构都不一样，没办法用一种“线性（*linear*）”结构来表达，如何在计算机中实现一个树形数据结构？面对这样的问题，我们应该如何思考呢？

一个优秀的起点是问题解决的一半。设计数据结构的起点是：**思考我们会怎么操作和使用这个数据结构**。

假定我们已经有个一个树形数据结构，我们可能会有这些操作场景（可以尝试用行政区划或者论坛版块等熟悉的实例来帮助思考）：
* 查找和遍历：
  * 给定一个 *node* 找出它的 *parent* ✪
  * 给定一个 *node* 找出它所有的 *children* ✪
  * 给定一个 *node* 找出它某个特定 *child*
  * 给定一个 *node* 找出它所有的 *siblings*
  * 给定一个 *node* 遍历其 *sub-tree* 即找出它的所有 *descendants* ✪
  * 给定 *node* *A* 和 *B*，找到 *A* 和 *B* 之间的 *path* 和 *distance*
  * 给定一个 *node* 确定它的 *tier*
* 编辑
  * 给一个 *node* 增加一个 *child* ✪
  * 删除一个 *node* ✪ -> 新一步思考：这意味着什么？删除整个子树还是？
  * 修改一个 *node* 的 *parent* ✪

差不多就这些，其中标记了 ✪ 号的是感觉特别重要和基础的操作。

通过形象化的图示、对基本概念的定义和表述、对可能操作场景的罗列，已经增加了很多我们对问题理解的深度和全面度，下来我们可以尝试做出一些初步设计判断了。

## 设计初步方案

从前面的分析，我们可以有这么一些收获：
* 树中节点之间的父子关系是核心和本质的内容；
* 表达这种父子关系的关键是节点，节点应保存父节点和子节点相关信息；
* 树的结构上带有显著的“递归”特点，可以有助于我们的设计。

如果你不记得“递归（*recursion*，*recursive*）”是怎么回事了，可以温习[递归函数](p2-4-recursion.ipynb)一章。

树也是递归的一个典型例子，因为一棵树可以看做由**根节点、根节点的子节点和以这些子节点为根的子树**合起来组成的，如果我们想实现对树进行操作的函数 `f()`，那么我们可以让这个函数接受一个树的根节点作为输入参数，这个函数大致上会是这个样子：

```python
def f(root):
    # 对 root 做一些操作
    # 然后取出 root 的所有子节点，对其中每个子节点调用 f() 本身
    for node in root.children:
        f(node)
```

也就是说，我们只要对某个节点做操作，然后获取这个节点的所有子节点，递归调用自己，就能对整个树做操作了。

在树形数据结构中，“获取一个节点的所有子节点”是至关重要的操作。

据此我们可以做出这么一个初步的设计：

In [1]:
class TreeNode:
    def __init__(self, name='root', data=None, parent=None, children=None):
        self.name = name
        self.data = data
        # 确认 parent 参数是 TreeNode 类型
        if perant:
            assert isinstance(parent, TreeNode)
            self.parent = parent
            self.children = []
        if children is not None:
            for child in children:
                self.add_child(child)

    def add_child(self, node):
        # 1. 确认 node 参数是 TreeNode 类型
        # 2. 将要加入的子节点的 parent 属性设为自己
        # 3. 然后将其加入 children 列表
        assert isinstance(node, TreeNode)
        node.parent = self
        self.children.append(node)

我们设计的核心数据结构是表示树节点的自定义类型 TreeNode，这个类型的对象有四个实例变量（属性）：
* *name*：节点的名字，最好能唯一标识出一个节点
* *data*：节点相关的任何数据，可以是任何数据类型
* *parent*：节点的父节点，如果没有父节点就是 None
* *children*：节点所有子节点组成的一个列表

这里面 *parent* 和 *children* 里的元素都必须是 TreeNode 类型的对象，我们在处理这两个属性时要先确认这一点，在上面的代码中我们用 Python 的 `assert` 语句和 `isinstance()` 函数来实现：
* `assert` 关键字后面的表达式必须返回 True，否则程序将抛出 `AssertionError` 异常后终止；
* `isinstance(obj, type)` 之前我们就介绍过，它接受两个参数，第一个参数是一个对象，而第二个参数是一个类型，函数判断第一个参数是不是第二个参数指明的类型，如果是返回 `True`，否则返回 `False`。

如上定义的 `TreeNode` 类，它实例化出来的对象 `node` 具备如下的能力：
* 很容易取得其父节点 *parent*；
* 很容易取得其所有子节点 *children*；
* 已经实现了增加子节点的操作。

最有意思的是，TreeNode 类型实际上也代表了树本身，因为一个节点加上它所有子节点，本来就是一棵树嘛！

在这个类型定义的基础上，我们可以实现前一节列出的所有操作，你可以把这作为练习动手试一试。

> 在实际项目中，我们并不需要定义树的数据结构，因为有优秀的第三方实现可用，比如 Python 非常棒的第三方库 [anytree](https://anytree.readthedocs.io/en/latest/#)，你可以试试，看和你自己实现的有什么区别。

## 小结

这一章介绍非常常见的树形逻辑怎么实现。没有很多代码，主要讲的是思维方法，请你读完一遍之后尝试自己从头思考和建立一个基本的树的数据结构，如果遇到问题就再读一遍。

In [2]:
class TreeNode:
    def __init__(self, name='root', data=None, parent=None, children=None):
        self.name = name
        self.data = data
        if parent:
            assert isinstance(parent, TreeNode)
            parent.add_child(self)
        self.parent = parent
        self.children = []
        if children is not None:
            for child in children:
                self.add_child(child)

    def add_child(self, node):
        assert isinstance(node, TreeNode)
        node.parent = self
        self.children.append(node)
        
    def add_children(self, *nodes):
        for node in nodes:
            assert isinstance(node, TreeNode)
            node.parent = self
            self.children.append(node)
            
    def remove(self, node):
        assert isinstance(node, TreeNode)            
        if node in self.children:
            for child in node.children:
                node.children.remove(child)
            self.children.remove(node)
        else:
            print('{node} is not child of {self}')
#还没想好应该如何在不知道node的parent（self代表的TreeNode实例）是谁的情况下实现‘先查找，再删除’的功能。
#如直接输入root.remove(a12) 而不是像写出来的method 写成a1.remove(a12)
#然后又引出新的问题，竟然可以让不同的TreeNode对应相同的变量(a和c都有child--c4)和(name="C4")
#出现上述问答题在使用self.find()时，应如何考虑？   
#比如‘吉林省---吉林市’ 应该在树的结构里name=‘吉林’ 不同的Tier对应‘省’、‘市’。
            
    
   
    def _get_parents(self, node, lst):
        if node.parent:
            lst.append(node.parent.name)
            root._get_parents(node.parent,lst)
        return lst

    def get_all_parents(self, node):
        lst = []
        return root._get_parents(node, lst)

    def get_tier(self,node):
        return len(root.get_all_parents(node)) +1
# 以root节点为tier=1 来定义 。
    
    
    def get_same_parents(self, node1, node2):
        lst1 ,lst2 = [node1.name],[node2.name]
        lst1,lst2 = root._get_parents(node1, lst1),root._get_parents(node2, lst2)
        for i in lst1:
            for n in lst2:
                if i == n:
                    print(f" '{node1.name}' and '{node2.name}' have same parent '{i}' ")
                    return i
    
    def get_distance(self, node1, node2):
        if (node1.name or node2.name) == root.get_same_parents(node1, node2):
            print(abs(root.get_tier(node1)-root.get_tier(node2)))
        else:
            lst1 ,lst2 = [node1.name],[node2.name]
            symmetric_difference_counts = len(set(root._get_parents(node1, lst1)) ^ set(root._get_parents(node2, lst2)))
            print(symmetric_difference_counts)
            
            

       
    def move(self, node, new_parent):
        assert isinstance(node, TreeNode)
        node.parent = new_parent.name
        new_parent.children.append(node)
        self.children.remove(node)
        
    def find(self, name):
        for child in self.children:
            if child.name == name:
                print(f"{self.name}'s child {name} found")
            else:
                if self.children and len(self.children):
                    child.find(name)
                    
    def get_siblings(self, node):
        assert isinstance(node, TreeNode)
        if len(node.parent.children) == 1:
            print(f'{node} has no sibling')
        else:
            for child in node.parent.children:
                if child != node:
                    print(f"{node.name}'s silblings found {child.name}")
                    
    def get_descendants(self, node):
        assert isinstance(node, TreeNode)
        if node.children:
            for child in node.children:
                print(f"found {node.name}'s descendant {child.name}")
                self.get_descendants(child)       
    
    def _to_strings(self, xs, _prefix='', _last=True):
        xs.append(''.join([_prefix, '|__ ' if _last else '|-- ', self.name]))
        _prefix += '    ' if _last else '|   '
        count = len(self.children)
        for n, node in enumerate(self.children):
            _last = n == (count-1)
            node._to_strings(xs, _prefix, _last)
    
    def __repr__(self):
        xs = [self.name]
        for node in self.children[:-1]:
            node._to_strings(xs, _last = False)
        if self.children:
#print——tree功能自己只能独立写到version 1.此为复制老师的代码/
#因为打印没有children的节点时，children时下一列中self.children[-1]报错out of range 。加入上一句判断//是否合适呢？？
            self.children[-1]._to_strings(xs, _last = True)
        return '\n'.join(xs)
    
    
    def __str__(self):
        return self.__repr__()
    
    
# 如method的运行和self无关的话，就用root作为Node了。如root.get_descendants(a)   root.get_siblings(b11)    root.find('C4')
# 相反remove的method需要制定Node。如c.move(c1,b)。请问这个一般是如何处理会比较好？
# 时候应该在所有的函数都要加上return（以便传给其他函数）和相应的print()（给出数据经函数处理后的描述性文字）？

In [3]:
root = TreeNode()
a = TreeNode(name='A', parent=root)
a1 = TreeNode(name='A1', parent=a)
a2 = TreeNode(name='A2', parent=a)
a11 = TreeNode(name='A11', parent=a1)
a12 = TreeNode(name='A12', parent=a1)
a13 = TreeNode(name='A13', parent=a1)
a21 = TreeNode(name='A21', parent=a2)
a22 = TreeNode(name='A22', parent=a2)
a23 = TreeNode(name='A23', parent=a2)
b = TreeNode(name='B', parent=root)
b1 = TreeNode(name='B1', parent=b)
b2 = TreeNode(name='B2', parent=b)
b11 = TreeNode(name='B11', parent=b1)
b12 = TreeNode(name='B12', parent=b1)
b13 = TreeNode(name='B13', parent=b1)
b21 = TreeNode(name='B21', parent=b2)
b22 = TreeNode(name='B22', parent=b2)
b23 = TreeNode(name='B23', parent=b2)
c = TreeNode(name='C', parent=root)
c1 = TreeNode(name='C1', parent=c)
c2 = TreeNode(name='C2', parent=c)
c11 = TreeNode(name='C11', parent=c1)
c12 = TreeNode(name='C12', parent=c1)
c13 = TreeNode(name='C13', parent=c1)
c21 = TreeNode(name='C21', parent=c2)
c3 = TreeNode(name='C3')
c4 = TreeNode(name='C4')

c.add_children(c3, c4)
c4 = TreeNode(name='C4', parent=a)





In [4]:

root

root
|-- A
|   |-- A1
|   |   |-- A11
|   |   |-- A12
|   |   |__ A13
|   |-- A2
|   |   |-- A21
|   |   |-- A22
|   |   |__ A23
|   |__ C4
|-- B
|   |-- B1
|   |   |-- B11
|   |   |-- B12
|   |   |__ B13
|   |__ B2
|       |-- B21
|       |-- B22
|       |__ B23
|__ C
    |-- C1
    |   |-- C11
    |   |-- C12
    |   |__ C13
    |-- C2
    |   |__ C21
    |-- C3
    |__ C4

In [5]:
root.get_distance(c, c13)



 'C' and 'C13' have same parent 'C' 
2


In [6]:
c1.name

'C1'

In [7]:
c11.name

'C11'

In [8]:
root.get_same_parents(c3, c11)


 'C3' and 'C11' have same parent 'C' 


'C'

In [9]:
if c11.name == root.get_same_parents(c3, c11):
    print(0)
else:
    print(1)

 'C3' and 'C11' have same parent 'C' 
1


In [12]:
root.get_all_parents(c11)

['C1', 'C', 'root']